# 00_config — 공통 설정/메타 스키마

In [7]:
import os
import sys
import json
from pathlib import Path
import pandas as pd
import numpy as np

In [8]:
# ---- 경로 ----

# 프로젝트 루트 기준 경로
ROOT = Path(r"C:\workspace\dacon_sri").resolve()

# 데이터 루트
DATA_BASE = ROOT / 'data' / 'internal' / '2. SRI시민패널조사'

# 분기별 파일 매핑
QUARTERS = {
    "Q3_2024": {
        "data": DATA_BASE / "2024년 3분기 패널조사" / "(공개용) 2024년 3분기 패널조사 데이터(가중치).xlsx",
        "codebook": DATA_BASE / "2024년 3분기 패널조사" / "2024년 3분기 패널조사 데이터_코드북.xlsx",
        "survey_pdf": DATA_BASE / "2024년 3분기 패널조사" / "2024년 3분기 패널조사 설문지(최종).pdf",
    },
    "Q4_2024": {
        "data": DATA_BASE / "2024년 4분기 패널조사" / "(공개용) 2024년 4분기 패널조사 데이터(가중치).xlsx",
        "codebook": DATA_BASE / "2024년 4분기 패널조사" / "2024년 4분기 패널조사 데이터_코드북.xlsx",
        "survey_pdf": DATA_BASE / "2024년 4분기 패널조사" / "2024년 4분기 패널 조사 설문지(최종).pdf",
    },
    "Q1_2025": {
        "data": DATA_BASE / "2025년 1분기 패널조사" / "(공개용) 2025년 1분기 패널조사 데이터(가중치).xlsx",
        "codebook": DATA_BASE / "2025년 1분기 패널조사" / "2025년 1분기 패널조사_코드북.xlsx",
        "survey_pdf": DATA_BASE / "2025년 1분기 패널조사" / "2025년 1분기 패널조사 설문지(최종).pdf",
    },
    "Q2_2025": {
        "data": DATA_BASE / "2025년 2분기 패널조사" / "(공개용) 2025년 2분기 패널조사 데이터(가중치).xlsx",
        "codebook": DATA_BASE / "2025년 2분기 패널조사" / "2025년 2분기 패널조사_코드북.xlsx",
        "survey_pdf": DATA_BASE / "2025년 2분기 패널조사" / "2025년 2분기 패널조사 설문지(최종).pdf",
    },
}

# ==== 분기 선택만, 여기서 바꾸세요 ====
CURRENT_QUARTER = "Q4_2024" # "Q3_2024" "Q4_2024" "Q1_2025" "Q2_2025"
# ====================================

# 경로 바인딩
DATA_PATH = QUARTERS[CURRENT_QUARTER]["data"]
CODEBOOK_PATH = QUARTERS[CURRENT_QUARTER]["codebook"]
QUESTIONNAIRE_PDF = QUARTERS[CURRENT_QUARTER]["survey_pdf"]

# 결과 저장 폴더 (분기별로 분리)
OUT_DIR = ROOT / 'output' / '2. SRI시민패널조사' / CURRENT_QUARTER
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 공통 유틸
def assert_exists(p: Path, name: str):
    if not p.exists():
        raise FileNotFoundError(f"[{name}] 경로 확인 필요: {p}")

assert_exists(DATA_PATH, "DATA_PATH")
assert_exists(CODEBOOK_PATH, "CODEBOOK_PATH")


# ---- 가중치 ----
WEIGHT_PRIORITY = ["wg","ws"]
def choose_weight(df: pd.DataFrame) -> pd.Series:
    for c in WEIGHT_PRIORITY:
        if c in df.columns:
            w = df[c].astype(float).fillna(1.0)
            return w
    return pd.Series(1.0, index=df.index, dtype=float)


# ---- eligibility & 통계 유틸 ----
def apply_eligibility(df: pd.DataFrame, base_col: str, rule_cfg: dict) -> pd.Series:
    if not rule_cfg or base_col not in df.columns:
        return pd.Series(True, index=df.index)
    rule = rule_cfg.get("rule","in")
    if rule == "in":
        return df[base_col].isin(rule_cfg.get("values", []))
    if rule == "in_top":
        return df[base_col].isin(rule_cfg.get("values_top", []))
    return pd.Series(True, index=df.index)


def cronbach_alpha(df_items: pd.DataFrame) -> float:
    d = df_items.dropna()
    k = d.shape[1]
    if k < 2 or d.shape[0] < 2:
        return float("nan")
    item_vars = d.var(axis=0, ddof=1)
    total_var = d.sum(axis=1).var(ddof=1)
    return float((k/(k-1)) * (1 - item_vars.sum()/total_var))

# ---------------- Var types (요약) ----------------
var_types = {
    "likert": {
        "description": "Likert 순서형 척도",
        "reverse": [],
        "scale": {"q1":7,"q2":7,"q3":7,"q5_1":7,"q5_2":7,"q5_3":7,"q5_4":7,"q5_5":7,"q7":7,"q18":5,"q19":5},
        "questions": ["q1","q2","q3","q5_1","q5_2","q5_3","q5_4","q5_5","q7","q18","q19"]
    },
    "ordinal-nonlikert": {
        "description": "순서형 비-Likert (증감·경향·구간·비교)",
        "reverse": ["q12"],
        "notes": {
            "q4":"기대 대비 성과('잘 모르겠다'=DK 별도 취급)",
            "q8":"의회 기대 대비 성과(동일)",
            "q12":"경기 전망(좋아짐↔나빠짐, 역코딩)",
            "q13":"지출 증감(늘어남↔줄어듦, 필요 시 이항화)",
            "DM4":"연령대 구간 서열",
            "DM10":"소득 구간 서열"
        },
        "questions": ["q4","q8","q12","q13","DM4","DM10"]
    },
    "ranked": {
        "description": "순위형(Top-K 선택, 점수화)",
        "rank_map": {"q3_1_1":1,"q3_1_2":2,"q3_1_3":3,"q11_1":1,"q11_2":2},
        "questions": ["q3_1_1","q3_1_2","q3_1_3","q11_1","q11_2"]
    },
    "categorical-nominal": {
        "description": "범주형-명목형",
        "questions": ["GU","q6","q9_2","q13_1","q15","q20","DM3","DM6","DM11","DM12","DM13","DM14","DM15"]
    }
}

# Likert 상세
LIKERT_7 = ["q1","q2","q3","q5_1","q5_2","q5_3","q5_4","q5_5","q7"]
LIKERT_5 = ["q18","q19"]  # 역코딩 필요
LIKERT_REVERSE = {"q18":5, "q19":5}
TNB_RULE = {7: {"bottom":[1,2], "neutral":[4], "top":[6,7]},
            5: {"bottom":[1,2], "neutral":[3], "top":[4,5]}}

# ordinal-nonlikert 상세

ONL_VARS = ["q4","q8","q12","q13","DM4","DM10"]
ONL_REVERSE = {"q12":5}
ONL_DK = {"q4":5,"q8":5}

# ranked 상세
RANK_VARS = ["q3_1_1","q3_1_2","q3_1_3","q11_1","q11_2"]
PRIORITY_WEIGHT = {1:3, 2:2, 3:1}

# categorical-nominal 상세 메타
categorical_nominal_detail = {
    "GU":   {"type":"region","encode":"onehot",
             "analysis":["weighted_dist","cross_q3","cross_q7"]},
    "q6":   {"type":"policy_priority","encode":"onehot",
             "notes":"개방형과 매핑시 테마 리프트 산출"},
    "q9_2": {"type":"companion","encode":"onehot",
             "eligibility":{"by":"q9","rule":"in","values":[1,2,4]}},
    "q13_1":{"type":"delta_amount_band","encode":"onehot",
             "notes":"서열성 있지만 분석상 명목 취급"},
    "q15":  {"type":"purchase_channel","encode":"onehot"},
    "q20":  {"type":"regulation_innovation_field","encode":"onehot"},
    "DM3":  {"type":"education","encode":"onehot"},
    "DM6":  {"type":"dual_income","encode":"onehot"},
    "DM11": {"type":"working_status_week","encode":"onehot"},
    "DM12": {"type":"has_child_yn","encode":"onehot"},
    "DM13": {"type":"job_experience","encode":"onehot"},
    "DM14": {"type":"occupation","encode":"onehot"},
    "DM15": {"type":"tenure_type","encode":"onehot"}
}

# compound items (q3_1, q5 bundle, q11)
compound_items = {
    "q3_1": {
        "children": ["q3_1_1","q3_1_2","q3_1_3"],
        "weights": {1:3, 2:2, 3:1},
        "eligibility": {"by":"q3","rule":"in_top","values_top":[5,6,7]},
        "outputs": {"scores": ["q3_1_1_score","q3_1_2_score","q3_1_3_score"], "sum": "q3_1_total_score"}
    },
    "q5_scale": {"children": ["q5_1","q5_2","q5_3","q5_4","q5_5"], "output":"q5_scale_mean"},
    "q11": {"children": ["q11_1","q11_2"], "weights": {1:3, 2:2},
            "outputs": {"scores":["q11_1_score","q11_2_score"], "sum":"q11_total_score"}}
}

# 타깃(결과) 변수
TARGET_ORDINAL = {"SAT_CITY":"q3", "SAT_COUNCIL":"q7"}

print('DATA_PATH exists:', DATA_PATH.exists())
print('CODEBOOK_PATH exists:', CODEBOOK_PATH.exists())
print('QUESTIONNAIRE_PDF exists:', QUESTIONNAIRE_PDF.exists())
print('OUT_DIR:', OUT_DIR)

DATA_PATH exists: True
CODEBOOK_PATH exists: True
QUESTIONNAIRE_PDF exists: True
OUT_DIR: C:\workspace\dacon_sri\output\2. SRI시민패널조사\Q4_2024
